In [0]:
from delta.tables import DeltaTable

# MERGE for incremental updates
deltaTable = DeltaTable.forName(spark, "workspace.default.events_table")
updates = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

In [0]:
deltaTable.alias("t").merge(
    updates.alias("s"),
    "t.user_session = s.user_session AND t.event_time = s.event_time"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

In [0]:
# Time travel
v0 = spark.read.format("delta").option("versionAsOf", 0).load("workspace.default.events_table")
yesterday = spark.read.format("delta") \
    .option("timestampAsOf", "2026-01-13").load("workspace.default.events_table")

# Optimize
spark.sql("OPTIMIZE events_table ZORDER BY (event_type, user_id)")
spark.sql("VACUUM events_table RETAIN 168 HOURS")

DataFrame[path: string]